In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount= True)

Mounted at /content/drive/


In [2]:
import keras
# !pip install gensim
# from gensim.models import word2vec
from os.path import join, exists, split
import os
import numpy as np

Using TensorFlow backend.


In [0]:
from keras.models import Sequential, load_model
cnnmodel = load_model('/content/drive/My Drive/Colab Notebooks/Trained models/CNN_onekernel.h5')

In [0]:
import numpy as np
import re
import itertools
from collections import Counter

def load_data_and_labels():
  positive_examples = list(open("/content/drive/My Drive/Colab Notebooks/Datasets/positive.csv").readlines())
  positive_examples = [s.strip() for s in positive_examples]
  negative_examples = list(open("/content/drive/My Drive/Colab Notebooks/Datasets/negative.csv").readlines())
  negative_examples = [s.strip() for s in negative_examples]
  x_text = positive_examples + negative_examples
  x_text = [s.split(" ") for s in x_text]
  positive_labels = [[0, 1] for _ in positive_examples]
  negative_labels = [[1, 0] for _ in negative_examples]
  y = np.concatenate([positive_labels, negative_labels], 0)
  return [x_text, y]

def pad_sentences(sentences, padding_word="<PAD/>"):
  sequence_length = max(len(x) for x in sentences)
  padded_sentences = []
  for i in range(len(sentences)):
      sentence = sentences[i]
      num_padding = sequence_length - len(sentence)
      new_sentence = sentence + [padding_word] * num_padding
      padded_sentences.append(new_sentence)
  return padded_sentences

def build_vocab(sentences):
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

def build_input_data(sentences, labels, vocabulary):
    x = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])
    y = np.array(labels)
    return [x, y]
  
  
def load_data():
    """
    Loads and preprocessed data for the MR dataset.
    Returns input vectors, labels, vocabulary, and inverse vocabulary.
    """
    # Load and preprocess data
    sentences, labels = load_data_and_labels()
    sentences_padded = pad_sentences(sentences)
    vocabulary, vocabulary_inv = build_vocab(sentences_padded)
    x, y = build_input_data(sentences_padded, labels, vocabulary)
    return [x, y, vocabulary, vocabulary_inv]
  
x, y, vocabulary, vocabulary_inv_list = load_data()

In [19]:
import random
positive_test = list(open("/content/drive/My Drive/Colab Notebooks/predict_accuracy/positive_test.csv").readlines())
positive_test = [s.strip() for s in positive_test]
negative_test = list(open("/content/drive/My Drive/Colab Notebooks/predict_accuracy/negative_test.csv").readlines())
negative_test = [s.strip() for s in negative_test]
test = positive_test + negative_test

positive_labels = [[0, 1] for _ in positive_test]
negative_labels = [[1, 0] for _ in negative_test]
y_test = np.concatenate([positive_labels, negative_labels], 0)
print(y_test.shape)
padding_word="<PAD/>"
sequence_length = 76
padded_sentences = []
for i in test:
  temp = i.split(' ')
  num_padding = sequence_length - len(temp)
  new_sentence = temp + [padding_word] * num_padding
  padded_sentences.append(new_sentence)


# random.shuffle(test)

x2 = []

for sentence in padded_sentences:
  x1 = []
  for word in sentence:
    if word in vocabulary:
      x1.append(vocabulary[word])
    else:
      x1.append(0)
  x2.append(x1)

x_test = np.array(x2)
y_test = np.array(y_test)
print(y_test.shape)

(20121, 2)
(20121, 2)


In [20]:
y_test.shape

(20121, 2)

In [0]:
y_test = y_test.argmax(axis=1)
shuffle_indices = np.random.permutation(np.arange(len(y_test)))
x_test = x_test[shuffle_indices]
y_test = y_test[shuffle_indices]

In [27]:
y_test.shape

(20121,)

In [28]:
cnnmodel.evaluate(x_test, y_test)

20121/20121 [==============================] - 3s 125us/step


[0.2910933465455767, 0.8959793250832464]